In [1]:
import pandas as pd
import openpyxl
import numpy as np
import seaborn as sns
import re

In [3]:
# Damos lectura al archivo attacks.csv

data = pd.read_csv('C:/Users/Oscar/Documents/Bootcamp Ironhack Labs/datamex_082020/module-1/pipelines-project/your-code/attacks.csv', encoding='latin-1')

## Case Number column 

In [4]:
# Se eliminan los valores en 0 de la columna Case Number para quedarnos con los valores completos del DataFrame

df = data.drop(data[data['Case Number']== '0'].index)

In [5]:
# De igual manera se eliminan las filas sin valores 

df = df.dropna(axis=0, subset=['Case Number'])

In [6]:
# por último se elimina la columna con valores xx en columna Case Number

df.drop(25722, axis = 0,inplace = True)

In [7]:
# Se opta por eliminar las columnas Case Number.1, Case Number.2, Unnamed: 22, Unnamed: 23, href formula porque sus elemntos son
# repetidos con las columnas Case Number y href,las Unnamed no contienen valores por lo que también fueron eliminadas

df.drop(['Case Number.1', 'Case Number.2', 'Unnamed: 22', 'Unnamed: 23', 'href formula'], axis=1, inplace=True)

In [8]:
df_2 = df.copy()

In [15]:
# Se eliminan las letras que acompañan 

df_2['Case Number'] = df_2['Case Number'].map(lambda x: x.rstrip('aAbBcC'))

## Year  &  original order columns

In [9]:
# Las columnas Year y original order son float y se convertirán en int por lo que se necesita convertir los valores NaN en 0

df_2[['Year', 'original order']] = df_2[['Year', 'original order']].fillna(0)

In [10]:
# Aplicamos una lambda en la columna Year para convertirlo en int

df_2['Year'] = df_2['Year'].apply(lambda x: int(x))

In [11]:
# De igual manera para la columna original order

df_2['original order'] = df_2['original order'].apply(lambda x: int(x))

## Activity column

In [12]:
# Se encuantra que el común entre las palabras sobre la actividad que se realizaba mientras ocurrieron lo ataques terminan en 
# ing, por lo que determiné imprimir únicamente las palabras con esta terminación

df_2['Activity']= df_2['Activity'].apply(str).str.findall(r"\b(\w+ing)\b")

In [13]:
# Se quitan los paréntesis de la columna Activity

df_2['Activity'] = df_2['Activity'].str.get(0)

In [14]:
# A los valores que se quedaron sin valor después del filtrado por terminación ing se les asigna un valor de Other ya que son 
# actividades varias. Decidí agruparlas en uno solo para un mejor manejo

df_2['Activity']= df_2['Activity'].fillna('Other')

In [17]:
# En todos los valores NaN de todas las columnas se coloca Unknown porque no se conoce el dato

df_2= df_2.fillna('Unknown')

## Name column

In [19]:
# Se coloca a los valores 'male' y 'female' de esta columna como nombre desconocido

df_2['Name'].replace(['male', 'female'], 'Unknown', inplace=True)

## Sex column

In [20]:
df_2['Sex '].replace(['M ', 'N'], 'M', inplace=True)

In [22]:
df_2['Sex '].replace(['.', 'lli'], 'Unknown', inplace=True)

## Fatal column

In [23]:
df_2['Fatal (Y/N)'].replace(['UNKNOWN', '2017'], 'Unknown', inplace=True)

In [24]:
df_2['Fatal (Y/N)'].replace([' N', 'N ', 'M'], 'N', inplace=True)

In [26]:
df_2['Fatal (Y/N)'].replace(['y'], 'Y', inplace=True)

## Species column

In [27]:
# Se observa como patrón la palabra shark y se toma la palabra antes de esta para delimitar la especie a la que pertenecen los
# tiburones

df_2['Species ']= df_2['Species '].apply(str).str.findall(r"\b(\w+\sshark)")

In [28]:
# Se eliminan los corchetes

df_2['Species '] = df_2['Species '].str.get(0)

In [29]:
# Valores NaN se coloca como Unknown

df_2['Species ']= df_2['Species '].fillna('Unknown')

In [30]:
df_2

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href,original order
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6303
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,Unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6302
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,Unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6301
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Unknown,M,Unknown,Minor injury to lower leg,N,Unknown,m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6300
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,diving,Gustavo Ramos,M,Unknown,Lacerations to leg & hand shark PROVOKED INCIDENT,N,Unknown,Tiger shark,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,Unknown,M,Unknown,FATAL,Y,Unknown,Unknown,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,6
6298,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Unknown,diving,Ahmun,M,Unknown,FATAL,Y,Unknown,Unknown,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5
6299,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,Unknown,FATAL,Y,Unknown,Unknown,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,4
6300,ND.0002,1883-1889,0,Unprovoked,PANAMA,Unknown,"Panama Bay 8ºN, 79ºW",Other,Jules Patterson,M,Unknown,FATAL,Y,Unknown,Unknown,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,3


In [33]:
data = df_2.to_csv('C:/Users/Oscar/Documents/Bootcamp Ironhack Labs/datamex_082020/module-1/pandas-project/your-code/attacks.csv', index=False)